# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [1]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [5]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [8]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
# !wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
# !unzip dogImages.zip

In [3]:
import os 

In [6]:
role = get_execution_role()
bucket='course4-project'

In [26]:
from io import BytesIO
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [8]:
folder_prefix = "dogImages/train/"
bucket_name = 'course4-project'

In [9]:
s3_client = boto3.client('s3')

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [27]:
#TODO: Declare your HP ranges, metrics etc.
from sagemaker.tuner import (
        CategoricalParameter, 
        ContinuousParameter, 
        HyperparameterTuner, 
        IntegerParameter
)

hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(0.001, 0.1),
    'batch_size': CategoricalParameter([16 ,32, 64, 128, 256])
}

objective_metric_name = 'average test loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'average test loss', 'Regex': 'Test Average Loss: ([0-9\\.]+)'}]

In [28]:
#TODO: Create estimators for your HPs

estimator = PyTorch(
    entry_point = 'hpo.py',
    base_job_name = 'dog-breed-classification-hyperparameter',
    role = role,
    instance_count = 1,
    instance_type = 'ml.m5.2xlarge',
    framework_version = '1.9',
    py_version = 'py38'
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    objective_type=objective_type,
    max_jobs=4,
    max_parallel_jobs=1
)   

In [29]:
data_path = f's3://{bucket}/dogImages/'
data_path

's3://course4-project/dogImages/'

In [30]:
# TODO: Fit your HP Tuner
tuner.fit({'train': data_path}, wait=True) # TODO: Remember to include your data channels

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


............................................................................................................................................!


In [31]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()


2024-11-17 20:03:45 Starting - Found matching resource for reuse
2024-11-17 20:03:45 Downloading - Downloading the training image
2024-11-17 20:03:45 Training - Training image download completed. Training in progress.
2024-11-17 20:03:45 Uploading - Uploading generated training model
2024-11-17 20:03:45 Completed - Resource reused by training job: pytorch-training-241117-1954-004-c4d79719


{'_tuning_objective_metric': '"average test loss"',
 'batch_size': '"256"',
 'learning_rate': '0.03620856736835261',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"dog-breed-classification-hyperparameter-2024-11-17-19-54-05-462"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-982822995001/dog-breed-classification-hyperparameter-2024-11-17-19-54-05-462/source/sourcedir.tar.gz"'}

In [32]:
best_hyperparameters = {'batch_size': 256, 'learning_rate': 0.03620856736835261}

In [33]:
tuner.best_training_job()

'pytorch-training-241117-1954-003-e6cf76f1'

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
from sagemaker.debugger import (
    Rule, 
    rule_configs, 
    ProfilerRule, 
    DebuggerHookConfig, 
    CollectionConfig, 
    ProfilerConfig, 
    FrameworkProfile
)

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

hook_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()